In [112]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configuración del splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño del fragmento en caracteres (ajusta según tu necesidad)
    chunk_overlap=100  # Superposición para mantener contexto entre fragmentos
)


In [114]:
max_length = 0
max_article = ""

for index, row in df.iterrows():
    article_text = row['Texto_articulo']
    article_length = len(article_text)  # Obtener la longitud del artículo

    if article_length > max_length:
        max_length = article_length
        max_article = article_text  # Guardar el artículo más largo

# Mostrar el artículo más largo y su longitud
print(f"El artículo más largo tiene {max_length} caracteres.")
print(f"Texto del artículo más largo: {max_article[:100]}...")

El artículo más largo tiene 14324 caracteres.
Texto del artículo más largo: El S Ley N° 19.496 Art. 58 D.O. 07.03.1997 ervicio Nacional del Consumidor deberá velar por el cumpl...


In [125]:
import pandas as pd
from langchain_core.documents import Document

df = pd.read_csv("Ley_consumidor.csv")
# Crear documentos y dividir los artículos largos
documents = []
for index, row in df.iterrows():
    article_text = row['Texto_articulo']
    
    # Solo dividir si el artículo es más largo que el tamaño de chunk
    if len(article_text) > 1000:
        splits = splitter.split_text(article_text)
        for split in splits:
            documents.append(
                Document(
                    page_content=split,
                    metadata={'Articulo': row['Articulo']}
                )
            )
    else:
        # Si no es largo, añadirlo sin dividir
        documents.append(
            Document(
                page_content=article_text,
                metadata={'Articulo': row['Articulo']}
            )
        )

In [126]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")


vector_1 = embeddings.embed_query(documents[0].page_content)
vector_2 = embeddings.embed_query(documents[1].page_content)

assert len (vector_1) == len(vector_2)

print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.07067478, 0.06767536, -0.15741661, -0.024718808, -0.022629263, -0.042506766, -0.004066997, -0.05207465, -0.0013712747, 0.069366604]


In [127]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
len(vector_store.add_documents(documents))


354

In [128]:
ids = vector_store.add_documents(documents=documents)

In [129]:
results = vector_store.similarity_search(
    "Según el Artículo 46 de la Ley N° 19.496, ¿qué debe hacer un fabricante, importador, distribuidor o prestador de servicios si detecta peligros no previstos en sus productos o servicios después de haberlos introducido al mercado?"
)
print(results[0])

page_content='Todo  Ley N° 19.496 Art. 46 D.O. 07.03.1997 fabricante, importador o distribuidor de bienes o prestador de servicios que, con posterioridad a la introducción de ellos en el mercado, se percate de la existencia de peligros o riesgos no previstos oportunamente, deberá ponerlos, sin demora, en conocimiento de la autoridad competente para que se adopten las medidas preventivas o correctivas que el caso amerite, sin perjuicio de cumplir con las obligaciones de advertencia a los consumidores señaladas en el artículo precedente.' metadata={'Articulo': 'Artículo 46.-'}


In [130]:
results_async = await vector_store.asimilarity_search(
    "Según el Artículo 46 de la Ley N° 19.496, ¿qué debe hacer un fabricante, importador, distribuidor o prestador de servicios si detecta peligros no previstos en sus productos o servicios después de haberlos introducido al mercado?"
)
print(results_async[0])

page_content='Todo  Ley N° 19.496 Art. 46 D.O. 07.03.1997 fabricante, importador o distribuidor de bienes o prestador de servicios que, con posterioridad a la introducción de ellos en el mercado, se percate de la existencia de peligros o riesgos no previstos oportunamente, deberá ponerlos, sin demora, en conocimiento de la autoridad competente para que se adopten las medidas preventivas o correctivas que el caso amerite, sin perjuicio de cumplir con las obligaciones de advertencia a los consumidores señaladas en el artículo precedente.' metadata={'Articulo': 'Artículo 46.-'}


In [121]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("Según el Artículo 46 de la Ley N° 19.496, ¿qué debe hacer un fabricante, importador, distribuidor o prestador")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.8219663837298038

page_content='Todo  Ley N° 19.496 Art. 46 D.O. 07.03.1997 fabricante, importador o distribuidor de bienes o prestador de servicios que, con posterioridad a la introducción de ellos en el mercado, se percate de la existencia de peligros o riesgos no previstos oportunamente, deberá ponerlos, sin demora, en conocimiento de la autoridad competente para que se adopten las medidas preventivas o correctivas que el caso amerite, sin perjuicio de cumplir con las obligaciones de advertencia a los consumidores señaladas en el artículo precedente.' metadata={'Articulo': 'Artículo 46.-'}


In [122]:
embedding = embeddings.embed_query("Según el Artículo 46 de la Ley N° 19.496, ¿qué debe hacer un fabricante, importador, distribuidor o prestador")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Todo  Ley N° 19.496 Art. 46 D.O. 07.03.1997 fabricante, importador o distribuidor de bienes o prestador de servicios que, con posterioridad a la introducción de ellos en el mercado, se percate de la existencia de peligros o riesgos no previstos oportunamente, deberá ponerlos, sin demora, en conocimiento de la autoridad competente para que se adopten las medidas preventivas o correctivas que el caso amerite, sin perjuicio de cumplir con las obligaciones de advertencia a los consumidores señaladas en el artículo precedente.' metadata={'Articulo': 'Artículo 46.-'}


In [131]:
from typing import List

from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query,k=1)

retriever.batch(
    [
        "Según el Artículo 46 de la Ley N° 19.496, ¿qué debe hacer un fabricante, importador, distribuidor o prestador",
        "Según el Artículo 2 de la Ley N° 19.496, ¿qué derechos tienen los consumidores en cuanto a la igualdad de trato por parte de los proveedores?"
    ]
)

[[Document(id='a7e81216-80cd-4b6f-bdda-cdbc3b46930c', metadata={'Articulo': 'Artículo 46.-'}, page_content='Todo  Ley N° 19.496 Art. 46 D.O. 07.03.1997 fabricante, importador o distribuidor de bienes o prestador de servicios que, con posterioridad a la introducción de ellos en el mercado, se percate de la existencia de peligros o riesgos no previstos oportunamente, deberá ponerlos, sin demora, en conocimiento de la autoridad competente para que se adopten las medidas preventivas o correctivas que el caso amerite, sin perjuicio de cumplir con las obligaciones de advertencia a los consumidores señaladas en el artículo precedente.')],
 [Document(id='19628b80-e9bb-41d7-b5a2-898bb95477cb', metadata={'Articulo': 'Artículo 14.-'}, page_content='Cuando Ley N° 19.496 Art. 14 D.O. 07.03.1997  con conocimiento del proveedor se expendan productos con alguna deficiencia usados o refaccionados o cuando se ofrezcan productos en cuya  Ley N° 19.955 Art. único N° 14  D.O. 14.07.2004 fabricación o elabo

In [124]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # puedes ajustar el número de resultados
)

questions = [
    "¿Qué establece el Artículo 2 sobre el trato igualitario a los consumidores?",
    "¿Qué debe hacer un proveedor si descubre defectos en un producto ya en el mercado?",
    "¿Qué derechos tiene un consumidor al comprar un producto usado?",
]

results = retriever.batch(questions)

for i, res in enumerate(results):
    print(f"\nPregunta {i+1}: {questions[i]}")
    for doc in res:
        print(f"- Artículo: {doc.metadata['Articulo']}")
        print(f"  Contenido: {doc.page_content[:300]}...\n")



Pregunta 1: ¿Qué establece el Artículo 2 sobre el trato igualitario a los consumidores?
- Artículo: Artículo 48.-
  Contenido: En el Ley N° 19.496 Art. 48 D.O. 07.03.1997  supuesto a que se refiere el inciso primero del artículo anterior, el proveedor de la mercancía deberá, a su costa, cambiarla a los consumidores por otra inocua, de utilidad análoga y de valor equivalente. De no ser ello posible, deberá restituirles lo qu...

- Artículo: Artículo 48.-
  Contenido: En el Ley N° 19.496 Art. 48 D.O. 07.03.1997  supuesto a que se refiere el inciso primero del artículo anterior, el proveedor de la mercancía deberá, a su costa, cambiarla a los consumidores por otra inocua, de utilidad análoga y de valor equivalente. De no ser ello posible, deberá restituirles lo qu...

- Artículo: Artículo 48.-
  Contenido: En el Ley N° 19.496 Art. 48 D.O. 07.03.1997  supuesto a que se refiere el inciso primero del artículo anterior, el proveedor de la mercancía deberá, a su costa, cambiarla a los consumi

Ahora implementaremos el modelo RAG